In [1]:
from ipynb.fs.full.lbp import EkstraksiCiri
from ipynb.fs.full.KNN import Klasifikasi
from ipynb.fs.full.KSVNN import Reduksi
from ipynb.fs.full.ExportCSV import ExportCSV
import numpy
import cv2
import matplotlib.pyplot as plt
import glob

In [2]:
class Skenario:
    def __init__(self, radius, tetangga, resize, zona, filename):
        self.lbp = EkstraksiCiri(radius, tetangga, resize, zona)
#         filename = "hasil_ekstraksi_ciri\lbp"+str(lbp)+"n"+str(tetangga)+"resize"+str(resize)+"zona"+str(zona)
        self.jumlah_fitur = (zona*zona*16)
        self.csv = ExportCSV(filename)
#         self.csv = ExportCSV("lbp"+str(lbp))
        self.folder = "hasil"
    
    def saveEkstraksiHeader(self):
        self.csv.saveHeader(self.jumlah_fitur)
        
    def saveKlasifikasiHeader(self, filename, header):
        self.csv = ExportCSV(filename)
        self.csv.saveHeader(header)
    
    def getDataClass(self, basePath):
        result = basePath.replace(self.folder, "")
        return result[1:]
    
    def startEkstraksi(self):
        basePathImage = self.folder+"/*"
        for classPath in glob.glob(basePathImage):
            dataClass = self.getDataClass(classPath)
            classPath = classPath + "\*.jpg"
            for data in glob.glob(classPath):
                result = [dataClass]
                image = cv2.imread(data)
                lbpResult = self.lbp.ekstrakImage(image)
                result.extend(lbpResult)
                self.csv.saveData(result)
    
    def startKlasifikasi(self, k):
        knn = Klasifikasi(k, 5)
        data, classData = self.csv.loadData()
        knn.procced(data, classData)
        result = [knn.score, knn.presisi, knn.recall, knn.time]
        return result
    
    def saveHasilKlasifikasi(self, filename, result):
        self.csv = ExportCSV(filename)
        self.csv.saveData(result)
    
    def startReduce(self, k):
        reduce = Reduksi(k, 5)
        dataset, datasetClass = self.csv.loadData()
        reduce.startSkenarioReduksi(dataset, datasetClass)
        result = [reduce.score, reduce.presisi, reduce.recall, reduce.time]
        return result

In [3]:
# header_klasifikasi = ["k=1", "presisi", "recall", "time", "k=3", "presisi", "recall", "time",
#                       "k=5", "presisi", "recall", "time", "k=7", "presisi", "recall", "time", "k=9", "presisi", "recall", "time"]
# klasifikasi = ExportCSV("hasil_klasifikasi.csv")
# klasifikasi.createFile(header_klasifikasi)